In [1]:
from carla import Benchmark
from IPython.display import display
import carla.evaluation.catalog as evaluation_catalog
from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog
import shap 
from sklearn.preprocessing import OrdinalEncoder
from carla.data.causal_model import CausalModel
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from carla.data.catalog import CsvCatalog
import numpy as np 

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2


/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [2]:
scm = CausalModel("adult")
dataset = scm.generate_dataset(1000)

display(dataset.df)

Adult Model Initiate
Structural Equation Finished
Noise Distribution Finished


,label,x1,x2,x3,x7,x8,x4,x5,x6,x9
0,1.0,0.209713,7.964722,72.674016,9.218098,0.746881,3.431864,36.655200,24138.543295,970.841102
1,1.0,0.621170,32.283520,71.136914,7.775520,8.822875,4.426474,84.552880,67025.121640,2686.704442
2,1.0,0.419018,22.434469,22.204955,2.164510,0.657294,2.740916,68.681644,40335.455243,1617.015752
3,1.0,0.469601,38.217414,43.312022,6.762862,10.355841,3.087663,67.246334,99837.120591,3997.210231
4,1.0,0.104522,38.343006,50.139465,2.658391,4.636118,2.983128,22.333923,39954.090669,1602.077175
...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.543491,29.134157,86.830254,3.865837,6.666050,4.350597,98.722421,12237.188194,496.988502
996,1.0,0.052610,6.625969,60.254544,6.668043,10.393771,1.997369,39.754778,30228.365562,1213.470274
997,1.0,0.420042,20.528051,38.367934,5.260109,2.035357,0.429487,15.068320,97635.638318,3907.596582
998,1.0,0.733500,8.285623,22.873297,6.538968,0.617192,3.205359,42.537857,24641.904131,988.069995


In [3]:
#Model 

training_params = {"lr": 0.01, "epochs": 3, "batch_size": 16, "hidden_size": [18, 9, 2]}

ml_model = MLModelCatalog(
    dataset, model_type="ann", load_online=False, backend="pytorch"
)
ml_model.train(
    learning_rate=training_params["lr"],
    epochs=training_params["epochs"],
    batch_size=training_params["batch_size"],
    hidden_size=training_params["hidden_size"],
    force_train=True
)


balance on test set 0.8546666666666667, balance on test set 0.872
Epoch 0/2
----------
train Loss: 85.4338 Acc: 0.1453

test Loss: 87.2000 Acc: 0.1280

Epoch 1/2
----------
train Loss: 85.4667 Acc: 0.1453

test Loss: 87.2000 Acc: 0.1280

Epoch 2/2
----------
train Loss: 85.4667 Acc: 0.1453

test Loss: 87.2000 Acc: 0.1280



In [4]:
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog

factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:20]

display(test_factual)

,label,x1,x2,x3,x7,x8,x4,x5,x6,x9
0,1.0,0.209713,7.964722,72.674016,9.218098,0.746881,3.431864,36.655200,24138.543295,970.841102
1,1.0,0.621170,32.283520,71.136914,7.775520,8.822875,4.426474,84.552880,67025.121640,2686.704442
2,1.0,0.419018,22.434469,22.204955,2.164510,0.657294,2.740916,68.681644,40335.455243,1617.015752
3,1.0,0.469601,38.217414,43.312022,6.762862,10.355841,3.087663,67.246334,99837.120591,3997.210231
4,1.0,0.104522,38.343006,50.139465,2.658391,4.636118,2.983128,22.333923,39954.090669,1602.077175
5,1.0,0.909648,38.723918,18.823968,4.833488,12.157063,2.381406,8.524222,56700.546251,2269.470277
6,0.0,0.436744,28.428212,84.962343,14.355066,8.573536,0.280389,18.932027,7338.862420,297.965523
7,1.0,0.694242,4.598859,45.442182,11.834673,8.142798,2.479838,88.031063,51689.630580,2072.270830
8,0.0,0.154327,14.054844,33.626855,8.515609,10.393373,2.043496,25.383566,29907.425521,1198.438019
9,1.0,0.611888,31.529391,80.564769,1.768969,12.515090,4.233183,8.969138,92170.413983,3691.387639


In [5]:
hyperparams = {"loss_type": "MSE", "binary_cat_features": False}

recourse_method = recourse_catalog.Wachter(ml_model, hyperparams)
cfs = recourse_method.get_counterfactuals(test_factual)
#from carla.models.negative_instances import predict_negative_instances
#from carla.recourse_methods.catalog.causal_recourse import (
#    CausalRecourse,
#    constraints,
#    samplers,
#)

#hyperparams = {
#    "optimization_approach": "brute_force",
#    "num_samples": 10,
#    "scm": scm,
#    "constraint_handle": constraints.point_constraint,
#    "sampler_handle": samplers.sample_true_m0,
#}
#cfs = CausalRecourse(ml_model, hyperparams).get_counterfactuals(test_factual)

display(cfs)

AttributeError: 'NoneType' object has no attribute 'get_feature_names'

In [ ]:
def get_pred_from_causal(scm,values,cf_label, mapping_dict):
    #TODO change everything hard coded
    values['target']=cf_label
    def _get_noise_string(node):
        def _get_node_id(node):
            return node[1:]
        if not node[0] == "x":
            raise ValueError
        return "u" + _get_node_id(node)
    exogenous_variables = np.concatenate(
        [
            np.array(values[mapping_dict[node]]).reshape(-1, 1)
            for node in scm.get_topological_ordering("exogenous")
        ],
        axis=1,
    )
    exogenous_variables = pd.DataFrame(
        exogenous_variables, columns=scm.get_topological_ordering("exogenous")
    )

    endogenous_variables = exogenous_variables.copy()
    endogenous_variables = endogenous_variables.rename(
        columns=dict(
            zip(
                scm.get_topological_ordering("exogenous"),
                scm.get_topological_ordering("endogenous"),
            )
        )
    )
    # used later to make sure parents are populated when computing children
    endogenous_variables.loc[:] = np.nan
    for node in scm.get_topological_ordering("endogenous"):
        parents = scm.get_parents(node)
        if endogenous_variables.loc[:, list(parents)].isnull().values.any():
            raise ValueError(
                "parents in endogenous_variables should already be occupied"
            )
        endogenous_variables[node] = scm.structural_equations_np[node](
            exogenous_variables[_get_noise_string(node)],
            *[endogenous_variables[p] for p in parents],
        )
    labels=endogenous_variables['x9'][0]
    return labels

In [ ]:
from carla.data import causal_model
from carla.evaluation import remove_nans
from carla.evaluation.api import Evaluation
class Sematic(Evaluation):
    """
    """

    def __init__(self, ml_model, causal_graph):
        self.ml_model= ml_model
        self.causal_graph=causal_graph
    def get_evaluation(self,factual: np.ndarray, counterfactual: np.ndarray, causal_model):
        # generate data 
        cf_label=self.ml_model.predict(np.array(counterfactual.values).reshape(1,-1))
        cf_label=np.argmax(cf_label)
        causal_label=get_pred_from_causal(self.causal_graph,counterfactual,cf_label, mapping_dict)
        print(cf_label)
        if cf_label ==causal_label:
            return 1 
        else: 
            return 0

class path_checker():
    pass


metric = Sematic(ml_model,scm)
result=metric.get_evaluation(np.array([]),df_cfs.iloc[0],mapping_dict)
''' 
Analysis
Probitlity Distributions 
Histogram True Positive, False Positives 
'''
'''
Parts of counterfactuals actually change 
1hop and 2 hop relations 
'''
'''Histogram Features / true Positive / false Positive'''
# first initialize the benchmarking class by passing
# black-box-model, recourse method, and factuals into it
#benchmark = Benchmark(ml_model, recourse_method, factuals)

# now you can decide if you want to run all measurements
# or just specific ones.
#evaluation_measures = [
#    evaluation_catalog.YNN(benchmark.mlmodel, {"y": 5, "cf_label": 1}),
#    evaluation_catalog.Distance(benchmark.mlmodel),
#    evaluation_catalog.SuccessRate(),
#    evaluation_catalog.Redundancy(benchmark.mlmodel, {"cf_label": 1}),
#    evaluation_catalog.ConstraintViolation(benchmark.mlmodel),
#    evaluation_catalog.AvgTime({"time": benchmark.timer}),
#]

# now run all implemented measurements and create a
# DataFrame which consists of all results
#results = benchmark.run_benchmark(evaluation_measures)


NameError: name 'df_cfs' is not defined